In [1]:
import minsearch
import json

In [2]:
with open('documents.json', 'rt') as f:
    docs_raw = json.load(f)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
index.fit(documents)

In [7]:
q = "the course has already started, can I still join?"

In [8]:
def search(query):
    # Means the question field is 3x more important 
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"}, # only search in this course
        boost_dict=boost,
        num_results=5 # return 5 results
    )

    return results

In [9]:
results = search(q)
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [10]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

client = OpenAI(
  base_url=os.getenv("OPENAI_BASE_URL"),
  api_key=os.getenv("OPENAI_API_KEY")
)

In [11]:
def get_completion(prompt, model=os.getenv("OPENAI_MODEL")):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [12]:
def llm(prompt):
    response = get_completion(prompt)

    return response

In [13]:
def build_prompt(query, search_results):
    context = ""

    for doc_dict in search_results:
        for k, v in doc_dict.items():
            context += f"{k}: {v}\n"

    prompt = f"""
        You're a course teaching assistant. Answer the question based on the context. 
        Use only the facts from the context when answering the question. 
        If the context does not contain enough information to answer the question, say "I don't know".

        Question: {query}
        Context: {context}
    """
    return prompt.strip()

In [14]:
def rag(question):
    # Search for relevant documents
    search_results = search(question)
    
    # Build a prompt with the question and search results
    prompt = build_prompt(question, search_results)
    
    # Get the answer from the LLM
    answer = llm(prompt)
    
    return answer

In [15]:
question = "is the course free?"
rag(question)

"I don't know. The provided context does not mention whether the course is free or has any associated costs. The information focuses on registration details, prerequisites, deadlines, and post-course activities, but there is no explicit statement about the course's pricing structure."

In [16]:
from elasticsearch import Elasticsearch

docker run -it --rm --name elasticsearch \
  -p 19200:9200 \
  -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  elasticsearch:8.4.3


In [17]:
es_client = Elasticsearch("http://localhost:19200")

In [18]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
# in elasticsearch, index is like a table in a database
# documents are like rows in a table
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [19]:
from tqdm.auto import tqdm

In [20]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [21]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # question field is 3x more important
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    es_results = []
    for hit in response['hits']['hits']:
        es_results.append(hit['_source'])

    return es_results

In [22]:
def rag_es(question):
    # Search for relevant documents
    search_results = elastic_search(question)
    
    # Build a prompt with the question and search results
    prompt = build_prompt(question, search_results)
    
    # Get the answer from the LLM
    answer = llm(prompt)
    
    return answer

In [23]:
question = "Can i still join after the course started?"
rag_es(question)

'Yes, you can join the course after it has started. The course materials remain available, and you can follow the content at your own pace. You are eligible to submit homeworks even without prior registration. However, be mindful of deadlines for the final projects to avoid last-minute issues.'